# Kafka as a Platform
This notebook contains my notes from the *Kafka as a Platform* talk by Gary Berglund. Gary works for [Confluent](https://www.confluent.io/), which is the "Kafka company". 

Among other things, Confluent provides a managed Kafka service. Tim said this about operating Kafka: "Nobody loves that". AWS also has a new managed Kafka service too.

I went to this session to get a high-level overview of what Kafka provides these days. I looked at Kafka back in 2014 but never did use it.

### Kafka Basics
##### Events
In Kafka, the most basic data model is the *event*, which is a key/value pair. Kafka itself doesn't care about the types of the key and value, but there are higher-level schemas that you can layer on top of Kafka.

Tim gave some examples of common events people put in Kafka:

* IoT sensor reading
* Customer interaction
* Business process change
* Microservice output

##### Topics
Kafka puts events in structures called *topics*. A topic is an ordered collection of events. It is a *commit log*; you can't insert an event in the middle, delete an event, etc. This means events are *immutable*. You can scan the list and read the events sequentially. 

A Kafka topic is not a *queue*. Queues allow you to take messages out and delete them. Kafka is a *commit log*, so it is a durable record of events that can be read as many times as needed by many different consumers. Kafka does have a configurable retention period for events, because for large amounts of data you obviously can't keep everything forever.

Kakfa supports constant-time reads and writes. Reads are not very fancy; there are no complex message brokers that support joining topics, etc.

##### Partitions
Topics are split into *partitions*. These partitions are actual files sitting on one of the Kafka brokers (servers). Partitions are replicated on multiple brokers. Partitioning across multiple instances means it provides scalable writes and storage.

Pretty typical pattern in distributed systems used to run a hash function on the key to assign an event to a partition on the topic. This means that it's really only individual paritions that are guaranteed ordered. So you can count on all events of the same key being ordered.  

Because of how only a single partition is ordered, but multiple partitions are not, key selection becomes an important concern.

### Kafka Producers
A *producer* is an application that puts messages into topics. In Java, there is a KafkaProducer class that you can use to produce events. It handles partitioning and network protocol. Avro is the serialization protocol used.

There are also Kafka libraries for many other lanagues; they seem to be community-driven and lag a bit behind the Java library in new feature support.

### Kakfa Consumers
A *consumer* is an application that reads messages from topics. It has to get messages from multiple partitions. Time said: "Producers are easy, consumers are complicated".

Consuming a message doesn't make it go away. You can read the same message from multiple consumers, as many times as you want, because messages are durable. Consumers each keep track of their own offsets, so they don't conflict with each other. Offsets are stored in Kafka, so you don't need to worry about keeping that state yourself.

Having to scale a single consumer works by having each consumer looking at a single partition. If you need to scale consumption of a topic, it needs to be partitioned at least as much as you have instances of your consumer. A single partition can have *at most* 1 unique consumer. This means if you need to scale your consumption of a topic, you may need to re-partition the topic first.

Tim claims that using a commit log architecture like Kakfa unlocks a lot of architectural options for building evolutionary architectures. You can materialize whatever views you want from the immutable events.

### Kafka Connect
Speaker said that the low-level KafkaProducer and KafkaConsumer classes are like "Kafka assmembly language". Sometimes it works, but usually higher-level interfaces are better.

*Kafka Connect* is the ecosystem around Kafka that allows you to connect things like databases, salesforce, etc. to Kafka. It is a separate cluster that runs outside Kafka that helps you move things to/from Kakfa.

Connectors curated by Confluent are available at [hub.confluent.io](https://www.confluent.io/hub/). Some are open-source, some are commercial. You can write your own connector,  but most things should already be supported by community.

### Schema Registry
Kafka treats events as untyped, but consumers care about the type of events. Schema management is important since you'll have many consumers over many events over time.

The schema registry keeps track of versions of object schemas. The producer talks to the schema registry and registers a new schema before it can publish an event. The consumer reads the event and needs to know whether it can process the version of the object.

Schemas are stored and managed as JSON files.

### Kafka Streams
Tim said there are several common tasks people do with Kafka consumers:

* Aggregate streams
* Turn streams into lookup tables
* Enrich another stream with that table
* Join one stream with another

For these common use cases, Kafka provides a higher-level interface called *Kafka Streams*. It is a functional Java API that provides higher-level abstractions for streams and tables. 

Tim said this should be used over the low-level consumer and producer classes whenever possible.

### KSQL
KSQL is exactly what it sounds like: A SQL-like language that operates on Kafka streams and tables. KSQL is built on top of Kafka Streams.

KSQL runs in a separate cluster outside of Kafka. Results from the query are streams, which means that the results are continually being produced. Results go to another topic.

### Takeaways
As a result of attending this session, I would like to do the following:

* Build a small test application that uses Kafka (probably the high-level Streams API and a Kafka Connect connector)
* Find out what pieces discussed today are open-source and what pieces are commercial Confluent products.
* Look at the Kafka usage in existing applications at work.